In [1]:
import os
import urllib.request as request
from zipfile import ZipFile
from collections import namedtuple


from deepclassifier.constants import *
from deepclassifier.utils.common import read_yaml, create_directories


In [2]:
os.chdir("../")

In [3]:
from dataclasses import dataclass
@dataclass(frozen=True)
class BaseModelCreationConfig:
    root_dir : Path
    base_model_path : Path
    updated_base_model_path : Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int
    

In [4]:
from pytest import param


class BaseModelConfiguratorManager:
    def __init__(self,
        config_file_path=CONFIG_FILE_PATH,params_file_path=PARAMS_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])

    def prepare_base_model_creation_config(self)->BaseModelCreationConfig:
        config=self.config.prepare_base_model
        create_directories([config.root_dir])

        prepare_base_model_config=BaseModelCreationConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
             
        )
        return prepare_base_model_config

In [5]:
import tensorflow as tf

class PrepareBaseModel:
    def __init__(self, config: BaseModelCreationConfig):
        self.config = config    

    def get_base_model(self):
        self.model= tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )
        self.savemodel(path=self.config.base_model_path,model=self.model)

    @staticmethod
    def _prepare_full_model(model,classes,freeze_all,freeze_till,learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable=False
        elif (freeze_till is not None) and freeze_till >0:
            for layer in model.layers[:-freeze_till]:
                model.trainable=False

        flatten_in=tf.keras.layers.Flatten()(model.output)
        prediction=tf.keras.layers.Dense(
            units=classes,
            activation="softmax")(flatten_in)

        full_model=tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        
        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model


    def update_base_model(self):
        self.full_model= self._prepare_full_model(model=self.model, 
            classes=self.config.params_classes,
            freeze_all=True,freeze_till=None, 
            learning_rate=self.config.params_learning_rate) 

        self.savemodel(path=self.config.updated_base_model_path,model=self.full_model)

    @staticmethod
    def savemodel(path: Path, model: tf.keras.Model):
        model.save(path)

In [6]:
try:
    config = BaseModelConfiguratorManager()
    prepare_base_model_config = config.prepare_base_model_creation_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2022-11-02 16:30:22,998: INFO: common]: yaml file: configs\config.yaml loaded successfully
[2022-11-02 16:30:23,002: INFO: common]: yaml file: params.yaml loaded successfully
[2022-11-02 16:30:23,006: INFO: common]: created directory at: artifacts
[2022-11-02 16:30:23,008: INFO: common]: created directory at: artifacts/prepare_base_model
[2022-11-02 16:30:24,517: WARNING: saving_utils]: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.


AttributeError: 'Functional' object has no attribute 'ouput'